In [1]:
import sympy as sp

In [2]:
om1,om2,om3,omR,omas,k1,k2,k3,kR,kas = sp.symbols('omega_1 omega_2 omega_3 Omega_R omega_as k_1 k_2 k_3 k_R k_as',real=True)
t,z,q = sp.symbols('t z q', real=True)
pol_ten, eps0 = sp.symbols('\\left(\\frac{\\partial\\alpha}{\\partial{q}}\\right)_0 epsilon_0')
T2 = sp.symbols('T_2', real=True)
Q = sp.Function('Q',real=True)(z,t)

In [3]:
q_expr = sp.I/2*Q*sp.exp(sp.I*(kR*z-omR*t))
#q_expr = q_expr + sp.conjugate(q_expr)
sp.Eq(q, q_expr)

Eq(q, I*Q(z, t)*exp(I*(-Omega_R*t + k_R*z))/2)

In [4]:
dqdt_expr = sp.diff(q_expr,t,1)
dqdt_expr

Omega_R*Q(z, t)*exp(I*(-Omega_R*t + k_R*z))/2 + I*exp(I*(-Omega_R*t + k_R*z))*Derivative(Q(z, t), t)/2

In [5]:
d2qdt2_expr = sp.diff(q_expr,t,2)
d2qdt2_expr

(-I*Omega_R**2*Q(z, t)/2 + Omega_R*Derivative(Q(z, t), t) + I*Derivative(Q(z, t), (t, 2))/2)*exp(-I*(Omega_R*t - k_R*z))

In [6]:
#dampped harmonic oscillator
lhs = d2qdt2_expr + 2/T2*dqdt_expr + omR**2*q
lhs.expand()

Omega_R**2*q - I*Omega_R**2*Q(z, t)*exp(-I*Omega_R*t)*exp(I*k_R*z)/2 + Omega_R*exp(-I*Omega_R*t)*exp(I*k_R*z)*Derivative(Q(z, t), t) + Omega_R*Q(z, t)*exp(-I*Omega_R*t)*exp(I*k_R*z)/T_2 + I*exp(-I*Omega_R*t)*exp(I*k_R*z)*Derivative(Q(z, t), (t, 2))/2 + I*exp(-I*Omega_R*t)*exp(I*k_R*z)*Derivative(Q(z, t), t)/T_2

In [7]:
lhs_dev_by_exp = (lhs/sp.exp(sp.I*(kR*z-omR*t))).simplify().expand()
lhs_dev_by_exp.simplify().expand()

Omega_R**2*q*exp(I*Omega_R*t)*exp(-I*k_R*z) - I*Omega_R**2*Q(z, t)/2 + Omega_R*Derivative(Q(z, t), t) + Omega_R*Q(z, t)/T_2 + I*Derivative(Q(z, t), (t, 2))/2 + I*Derivative(Q(z, t), t)/T_2

In [8]:
#slowly varying ammplitude approximation
lhs_dev_by_exp_svaa = lhs_dev_by_exp.subs(sp.diff(Q,t,2),0)
lhs_dev_by_exp_svaa

Omega_R**2*q*exp(I*Omega_R*t)*exp(-I*k_R*z) - I*Omega_R**2*Q(z, t)/2 + Omega_R*Derivative(Q(z, t), t) + Omega_R*Q(z, t)/T_2 + I*Derivative(Q(z, t), t)/T_2

In [9]:
lhs_dev_by_exp_svaa_dev_by_omR = (lhs_dev_by_exp_svaa/omR).expand()
lhs_dev_by_exp_svaa_dev_by_omR

Omega_R*q*exp(I*Omega_R*t)*exp(-I*k_R*z) - I*Omega_R*Q(z, t)/2 + Derivative(Q(z, t), t) + Q(z, t)/T_2 + I*Derivative(Q(z, t), t)/(Omega_R*T_2)

In [10]:
#why the third term is neglected?
raman_mode_est = 100 #1/cm
T2_est = 1e-12
c_est = 3e10
inv_omR_T2 = 1/c_est/raman_mode_est/T2_est
inv_omR_T2

0.33333333333333337

In [11]:
lhs_dev_by_exp_svaa_dev_by_omR_real = lhs_dev_by_exp_svaa_dev_by_omR.subs(sp.I,0).expand()
lhs_dev_by_exp_svaa_dev_by_omR_real

Omega_R*q + Derivative(Q(z, t), t) + Q(z, t)/T_2

In [12]:
lhs_f = lhs_dev_by_exp_svaa_dev_by_omR_real*omR*sp.exp(-sp.I*(kR*z-omR*t))
lhs_f

Omega_R*(Omega_R*q + Derivative(Q(z, t), t) + Q(z, t)/T_2)*exp(-I*(-Omega_R*t + k_R*z))

In [13]:
x,y = sp.symbols('x y', real=True)
E10 = sp.Function('E_{10}')(t,x,y)
E20 = sp.Function('E_{20}')(t,x,y)
m = sp.symbols('m', real=True)
E1 = E10/2*sp.exp(-sp.I*(k1*z-om1*t))
E2 = E20/2*sp.exp(-sp.I*(k2*z-om2*t))
E = E1+sp.conjugate(E1)+E2+sp.conjugate(E2)
E

E_{10}(t, x, y)*exp(-I*(k_1*z - omega_1*t))/2 + E_{20}(t, x, y)*exp(-I*(k_2*z - omega_2*t))/2 + exp(I*(k_1*z - omega_1*t))*conjugate(E_{10}(t, x, y))/2 + exp(I*(k_2*z - omega_2*t))*conjugate(E_{20}(t, x, y))/2

In [14]:
E_drive = (E**2).expand().subs(E10*sp.conjugate(E20),1).subs(E10,0).subs(E20,0)*E10*sp.conjugate(E20)
E_drive

E_{10}(t, x, y)*exp(-I*k_1*z)*exp(I*k_2*z)*exp(I*omega_1*t)*exp(-I*omega_2*t)*conjugate(E_{20}(t, x, y))/2

In [15]:
rhs = pol_ten/m*E_drive
rhs_f = rhs.expand().simplify()
rhs_f

\left(\frac{\partial\alpha}{\partial{q}}\right)_0*E_{10}(t, x, y)*exp(I*(-k_1*z + k_2*z + omega_1*t - omega_2*t))*conjugate(E_{20}(t, x, y))/(2*m)

In [16]:
lhs_ff = lhs_f/sp.exp(-sp.I*(kR*z-omR*t))/omR*-1
lhs_ff

-Omega_R*q - Derivative(Q(z, t), t) - Q(z, t)/T_2

In [17]:
delom = sp.symbols('\\Delta{\\omega}')
rhs_ff = (rhs_f/sp.exp(-sp.I*(kR*z-omR*t))/omR*-1).expand().subs(sp.exp(sp.I*(om1-om2-omR)*t).expand(),sp.exp(sp.I*delom*t)).subs(sp.exp(-sp.I*(k1-k2)*z).expand(),sp.exp(-sp.I*kR*z))
rhs_ff

-\left(\frac{\partial\alpha}{\partial{q}}\right)_0*E_{10}(t, x, y)*exp(I*\Delta{\omega}*t)*conjugate(E_{20}(t, x, y))/(2*Omega_R*m)

In [18]:
#rhs_ff = (rhs_f/sp.exp(-sp.I*(kR*z-omR*t))/omR*-1).expand().subs(sp.exp(sp.I*(om1-om2)*t).expand(),sp.exp(sp.I*omR*t)).subs(sp.exp(-sp.I*(k1-k2)*z).expand(),sp.exp(-sp.I*kR*z))
#rhs_ff

In [19]:
##############################################################
##############################################################
##############################################################
#1_a
sp.Eq(lhs_ff,rhs_ff)

Eq(-Omega_R*q - Derivative(Q(z, t), t) - Q(z, t)/T_2, -\left(\frac{\partial\alpha}{\partial{q}}\right)_0*E_{10}(t, x, y)*exp(I*\Delta{\omega}*t)*conjugate(E_{20}(t, x, y))/(2*Omega_R*m))

In [20]:
#Anti-Stokes polarization
td,N = sp.symbols('t_d N',real=True)
E30 = sp.Function('E_{30}')(t,x,y)
E3 = E30.subs(t,t-td)*sp.exp(-sp.I*(k3*z-om3*(t-td)))
P_3_as=N*pol_ten*q*E3
P_3_as

N*\left(\frac{\partial\alpha}{\partial{q}}\right)_0*q*E_{30}(t - t_d, x, y)*exp(-I*(k_3*z - omega_3*(t - t_d)))

In [21]:
P_3_as_wo_omR = P_3_as.subs(omR,om1-om2)
P_3_as_wo_omR

N*\left(\frac{\partial\alpha}{\partial{q}}\right)_0*q*E_{30}(t - t_d, x, y)*exp(-I*(k_3*z - omega_3*(t - t_d)))

In [22]:
#Anti-Stokes polarization
Eas0 = sp.Function('E_{as0}')(z,t)
Eas = Eas0/2*sp.exp(-sp.I*(kas*z-omas*t))
Eas

E_{as0}(z, t)*exp(-I*(k_as*z - omega_as*t))/2

In [23]:
#wave equation, LHS
d2Edz2 = sp.diff(Eas,z,2)
d2Edz2

(-k_as**2*E_{as0}(z, t)/2 - I*k_as*Derivative(E_{as0}(z, t), z) + Derivative(E_{as0}(z, t), (z, 2))/2)*exp(-I*(k_as*z - omega_as*t))

In [24]:
d2Edt2 = sp.diff(Eas,t,2)
d2Edt2

(-omega_as**2*E_{as0}(z, t)/2 + I*omega_as*Derivative(E_{as0}(z, t), t) + Derivative(E_{as0}(z, t), (t, 2))/2)*exp(-I*(k_as*z - omega_as*t))

In [25]:
nas,c = sp.symbols('n_{as} c', real=True)
as_we_lhs = d2Edz2-(nas/c)**2*d2Edt2
as_we_lhs

(-k_as**2*E_{as0}(z, t)/2 - I*k_as*Derivative(E_{as0}(z, t), z) + Derivative(E_{as0}(z, t), (z, 2))/2)*exp(-I*(k_as*z - omega_as*t)) - n_{as}**2*(-omega_as**2*E_{as0}(z, t)/2 + I*omega_as*Derivative(E_{as0}(z, t), t) + Derivative(E_{as0}(z, t), (t, 2))/2)*exp(-I*(k_as*z - omega_as*t))/c**2

In [26]:
#slowly varying ammplitude approximation
as_we_lhs_svaa = as_we_lhs.subs(sp.diff(Eas0,t,2),0).subs(sp.diff(Eas0,z,2),0)
as_we_lhs_svaa

(-k_as**2*E_{as0}(z, t)/2 - I*k_as*Derivative(E_{as0}(z, t), z))*exp(-I*(k_as*z - omega_as*t)) - n_{as}**2*(-omega_as**2*E_{as0}(z, t)/2 + I*omega_as*Derivative(E_{as0}(z, t), t))*exp(-I*(k_as*z - omega_as*t))/c**2

In [27]:
as_we_lhs_svaa_subs_kas = as_we_lhs_svaa.subs(kas,nas*omas/c).simplify()
as_we_lhs_svaa_subs_kas

I*n_{as}*omega_as*(-c*Derivative(E_{as0}(z, t), z) - n_{as}*Derivative(E_{as0}(z, t), t))*exp(I*omega_as*(t - n_{as}*z/c))/c**2

In [28]:
as_we_lhs_f = as_we_lhs_svaa_subs_kas
as_we_lhs_f

I*n_{as}*omega_as*(-c*Derivative(E_{as0}(z, t), z) - n_{as}*Derivative(E_{as0}(z, t), t))*exp(I*omega_as*(t - n_{as}*z/c))/c**2

In [29]:
d2Pdt2 = sp.diff(P_3_as_wo_omR,t,2)
d2Pdt2

N*\left(\frac{\partial\alpha}{\partial{q}}\right)_0*q*(-omega_3**2*E_{30}(t - t_d, x, y) + 2*I*omega_3*Subs(Derivative(E_{30}(_xi_1, x, y), _xi_1), _xi_1, t - t_d) + Subs(Derivative(E_{30}(_xi_1, x, y), (_xi_1, 2)), _xi_1, t - t_d))*exp(-I*(k_3*z - omega_3*(t - t_d)))

In [30]:
d2Pdt2_svaa = d2Pdt2.subs(sp.diff(Q,t,2),0).subs(sp.diff(E30.subs(t,t-td),t,2),0)
d2Pdt2_svaa

N*\left(\frac{\partial\alpha}{\partial{q}}\right)_0*q*(-omega_3**2*E_{30}(t - t_d, x, y) + 2*I*omega_3*Subs(Derivative(E_{30}(_xi_1, x, y), _xi_1), _xi_1, t - t_d))*exp(-I*(k_3*z - omega_3*(t - t_d)))

In [31]:
as_we_rhs_svaa = (d2Pdt2_svaa/eps0/c**2).expand().subs(sp.diff(E30.subs(t,t-td),t,1),0).subs(sp.diff(Q,t,1),0).simplify()
as_we_rhs_svaa

-N*\left(\frac{\partial\alpha}{\partial{q}}\right)_0*omega_3**2*q*E_{30}(t - t_d, x, y)*exp(-I*(k_3*z - omega_3*t + omega_3*t_d))/(c**2*epsilon_0)

In [32]:
as_we_rhs_svaa_subs_R = as_we_rhs_svaa.subs(omR, om1-om2).subs(kR, k1-k2)
as_we_rhs_svaa_subs_R

-N*\left(\frac{\partial\alpha}{\partial{q}}\right)_0*omega_3**2*q*E_{30}(t - t_d, x, y)*exp(-I*(k_3*z - omega_3*t + omega_3*t_d))/(c**2*epsilon_0)

In [33]:
as_we_rhs_svaa_subs_R_sub_cons = as_we_rhs_svaa_subs_R.expand().subs(sp.exp(sp.I*(om1-om2+om3)*t).expand(),sp.exp(sp.I*omas*t)).simplify().subs(-((om1-om2+om3)**2).expand(),-omas**2)
as_we_rhs_f = as_we_rhs_svaa_subs_R_sub_cons
as_we_rhs_f

-N*\left(\frac{\partial\alpha}{\partial{q}}\right)_0*omega_3**2*q*E_{30}(t - t_d, x, y)*exp(-I*(k_3*z - omega_3*t + omega_3*t_d))/(c**2*epsilon_0)

In [34]:
as_we_lhs_f

I*n_{as}*omega_as*(-c*Derivative(E_{as0}(z, t), z) - n_{as}*Derivative(E_{as0}(z, t), t))*exp(I*omega_as*(t - n_{as}*z/c))/c**2

In [35]:
as_we_lhs_ff = (as_we_lhs_f/sp.exp(sp.I*omas*(t-nas*z/c))*c**2/sp.I/nas/omas/c*-1).simplify()
as_we_lhs_ff

Derivative(E_{as0}(z, t), z) + n_{as}*Derivative(E_{as0}(z, t), t)/c

In [36]:
delk = sp.symbols('\\Delta{k}', real=True)
as_we_rhs_ff = (as_we_rhs_f/sp.exp(sp.I*omas*(t-nas*z/c))*c**2/sp.I/nas/omas/c*-1).simplify().expand().subs(nas*omas/c,kas).simplify().subs(-((k1-k2+k3-kas)*z).expand(),delk*z)
as_we_rhs_ff

-I*N*\left(\frac{\partial\alpha}{\partial{q}}\right)_0*omega_3**2*q*E_{30}(t - t_d, x, y)*exp(-I*(k_3*z - k_as*z - omega_3*t + omega_3*t_d + omega_as*t))/(c*epsilon_0*n_{as}*omega_as)

In [37]:
##############################################################
##############################################################
##############################################################
#2_a
sp.Eq(as_we_lhs_ff,as_we_rhs_ff)

Eq(Derivative(E_{as0}(z, t), z) + n_{as}*Derivative(E_{as0}(z, t), t)/c, -I*N*\left(\frac{\partial\alpha}{\partial{q}}\right)_0*omega_3**2*q*E_{30}(t - t_d, x, y)*exp(-I*(k_3*z - k_as*z - omega_3*t + omega_3*t_d + omega_as*t))/(c*epsilon_0*n_{as}*omega_as))

In [38]:
#coordinate transform: t'= t - (z*nas/c) or t = t' + (z*nas/c), z'=z
zp,tp = sp.symbols('z\' t\'', real = True)
#d/dt -> d/dt' no change, just variable swap
sp.diff(tp+z*nas/c,tp,1)

1

In [39]:
#d/dz -> d/dz' no change, just variable swap
sp.diff(zp,zp,1)

1

In [40]:
lhs_ff_ref_chng = lhs_ff.subs(z,zp).subs(t,tp)
lhs_ff_ref_chng

-Omega_R*q - Derivative(Q(z', t'), t') - Q(z', t')/T_2

In [41]:
kappaQ = sp.symbols('\\kappa_Q')
kappaQ_sub = -pol_ten/2/omR/m
rhs_ff_ref_chng = rhs_ff.subs(z,zp).subs(t,tp)*kappaQ/kappaQ_sub
rhs_ff_ref_chng

\kappa_Q*E_{10}(t', x, y)*exp(I*\Delta{\omega}*t')*conjugate(E_{20}(t', x, y))

In [42]:
##############################################################
##############################################################
##############################################################
#1_b
sp.Eq(lhs_ff_ref_chng,rhs_ff_ref_chng)

Eq(-Omega_R*q - Derivative(Q(z', t'), t') - Q(z', t')/T_2, \kappa_Q*E_{10}(t', x, y)*exp(I*\Delta{\omega}*t')*conjugate(E_{20}(t', x, y)))

In [43]:
sp.Eq(kappaQ, kappaQ_sub)

Eq(\kappa_Q, -\left(\frac{\partial\alpha}{\partial{q}}\right)_0/(2*Omega_R*m))

In [44]:
#whay changing ref frame makes Eas0 only zp dependent?
sp.diff(sp.exp(-(tp+z*nas/c)**2),tp,1)

(-2*t' - 2*n_{as}*z/c)*exp(-(t' + n_{as}*z/c)**2)

In [45]:
as_we_lhs_ff_ref_chng = as_we_lhs_ff.subs(z,zp).subs(t,tp).subs(Eas0.subs(z,zp).subs(t,tp),sp.Function('E_{as0}')(zp)).simplify()
as_we_lhs_ff_ref_chng

Derivative(E_{as0}(z'), z')

In [46]:
kappaA = sp.symbols('\\kappa_A')
kappaA_sub = N*pol_ten*omas/2/c/eps0/nas
as_we_rhs_ff_ref_chng = as_we_rhs_ff.subs(z,zp).subs(t,tp)*kappaA/kappaA_sub
as_we_rhs_ff_ref_chng

-2*I*\kappa_A*omega_3**2*q*E_{30}(t' - t_d, x, y)*exp(-I*(k_3*z' - k_as*z' - omega_3*t' + omega_3*t_d + omega_as*t'))/omega_as**2

In [47]:
sp.Eq(kappaA,kappaA_sub)

Eq(\kappa_A, N*\left(\frac{\partial\alpha}{\partial{q}}\right)_0*omega_as/(2*c*epsilon_0*n_{as}))

In [48]:
##############################################################
##############################################################
##############################################################
#2_b
sp.Eq(as_we_lhs_ff_ref_chng,as_we_rhs_ff_ref_chng)

Eq(Derivative(E_{as0}(z'), z'), -2*I*\kappa_A*omega_3**2*q*E_{30}(t' - t_d, x, y)*exp(-I*(k_3*z' - k_as*z' - omega_3*t' + omega_3*t_d + omega_as*t'))/omega_as**2)

In [49]:
#Assuming
del_k_est = 0
om3t_est = 2*sp.pi
Q_est = sp.Function('Q')(tp) # Q is invarient of position
L = sp.symbols('L', real = True, positive=True) #sample length

In [50]:
lhs_ff_ref_chng_est = lhs_ff_ref_chng.subs(Q.subs(z,zp).subs(t,tp),Q_est)
lhs_ff_ref_chng_est

-Omega_R*q - Derivative(Q(t'), t') - Q(t')/T_2

In [51]:
rhs_ff_ref_chng_est = rhs_ff_ref_chng
rhs_ff_ref_chng_est

\kappa_Q*E_{10}(t', x, y)*exp(I*\Delta{\omega}*t')*conjugate(E_{20}(t', x, y))

In [52]:
##############################################################
##############################################################
##############################################################
#1_c
sp.Eq(lhs_ff_ref_chng_est,rhs_ff_ref_chng_est)

Eq(-Omega_R*q - Derivative(Q(t'), t') - Q(t')/T_2, \kappa_Q*E_{10}(t', x, y)*exp(I*\Delta{\omega}*t')*conjugate(E_{20}(t', x, y)))

In [53]:
as_we_lhs_ff_ref_chng_est = sp.Integral(as_we_lhs_ff_ref_chng).subs(zp,L)
as_we_lhs_ff_ref_chng_est

Integral(Derivative(E_{as0}(L), L), L)

In [54]:
as_we_rhs_ff_ref_chng_est = sp.Integral(as_we_rhs_ff_ref_chng.subs(delk,del_k_est).subs(om3*td,om3t_est).subs(Q.subs(z,zp).subs(t,tp),Q_est),(zp,0,L))
as_we_rhs_ff_ref_chng_est

Integral(-2*I*\kappa_A*omega_3**2*q*E_{30}(t' - t_d, x, y)*exp(-I*(k_3*z' - k_as*z' - omega_3*t' + omega_as*t' + 2*pi))/omega_as**2, (z', 0, L))

In [55]:
##############################################################
##############################################################
##############################################################
#2_c
sp.Eq(as_we_lhs_ff_ref_chng_est,as_we_rhs_ff_ref_chng_est)

Eq(Integral(Derivative(E_{as0}(L), L), L), Integral(-2*I*\kappa_A*omega_3**2*q*E_{30}(t' - t_d, x, y)*exp(-I*(k_3*z' - k_as*z' - omega_3*t' + omega_as*t' + 2*pi))/omega_as**2, (z', 0, L)))

In [56]:
#Eq 1_c solve using Greens function
#initial conditions
Q_at_neg_t0 = 0
H = sp.Function('H')(tp)
theta,t0 = sp.symbols('theta t_0', real = True)
Q_eval = sp.exp(-tp/T2)*sp.Integral(sp.exp(-theta/T2)*rhs_ff_ref_chng_est.subs(tp,theta)*H.subs(tp,tp-theta),(theta,-t0,sp.oo))
sp.Eq(Q.subs(Q,sp.Function('Q')(tp)),Q_eval)

Eq(Q(t'), exp(-t'/T_2)*Integral(\kappa_Q*E_{10}(theta, x, y)*H(t' - theta)*exp(-theta/T_2)*exp(I*\Delta{\omega}*theta)*conjugate(E_{20}(theta, x, y)), (theta, -t_0, oo)))

In [57]:
##############################################################
##############################################################
##############################################################
#3
Eas0_eval = as_we_rhs_ff_ref_chng_est.subs(Q.subs(Q,sp.Function('Q')(tp)),Q_eval)
sp.Eq(as_we_lhs_ff_ref_chng_est,Eas0_eval)

Eq(Integral(Derivative(E_{as0}(L), L), L), Integral(-2*I*\kappa_A*omega_3**2*q*E_{30}(t' - t_d, x, y)*exp(-I*(k_3*z' - k_as*z' - omega_3*t' + omega_as*t' + 2*pi))/omega_as**2, (z', 0, L)))

In [58]:
##############################################################
##############################################################
##############################################################
#4
Ias = sp.Function('I_{as}')(L,tp,x,y)
Ias_eval = eps0*c*nas*sp.Abs(Eas0.subs(z,L).subs(t,tp))**2/2
sp.Eq(Ias,Ias_eval)

Eq(I_{as}(L, t', x, y), c*epsilon_0*n_{as}*Abs(E_{as0}(L, t'))**2/2)

In [59]:
##############################################################
##############################################################
##############################################################
#5
Ias_eval2 = eps0*c*nas*sp.Abs(Eas0_eval)**2/2
sp.Eq(Ias,Ias_eval2)

Eq(I_{as}(L, t', x, y), c*epsilon_0*n_{as}*Abs(Integral(-2*I*\kappa_A*omega_3**2*q*E_{30}(t' - t_d, x, y)*exp(-I*(k_3*z' - k_as*z' - omega_3*t' + omega_as*t' + 2*pi))/omega_as**2, (z', 0, L)))**2/2)

In [60]:
#Pulse energy
Xias = sp.Function('\\xi_{as}')(td)
Xias_eval = sp.Integral(sp.Integral(sp.Integral(Ias,(tp,-sp.oo,sp.oo)),(x,-sp.oo,sp.oo)),(y,-sp.oo,sp.oo))
sp.Eq(Xias,Xias_eval)

Eq(\xi_{as}(t_d), Integral(I_{as}(L, t', x, y), (t', -oo, oo), (x, -oo, oo), (y, -oo, oo)))

In [61]:
Xias_eval2 = sp.Integral(sp.Integral(sp.Integral(Ias_eval2,(tp,-sp.oo,sp.oo)),(x,-sp.oo,sp.oo)),(y,-sp.oo,sp.oo))
sp.Eq(Xias,Xias_eval2)

Eq(\xi_{as}(t_d), Integral(c*epsilon_0*n_{as}*Abs(Integral(-2*I*\kappa_A*omega_3**2*q*E_{30}(t' - t_d, x, y)*exp(-I*(k_3*z' - k_as*z' - omega_3*t' + omega_as*t' + 2*pi))/omega_as**2, (z', 0, L)))**2/2, (t', -oo, oo), (x, -oo, oo), (y, -oo, oo)))

In [62]:
#Introdusing pulsed fields
A10, A20, A30 = sp.symbols('A_{10}, A_{20}, A_{30}',real=True, positive=True)
tau, w0 = sp.symbols('tau, w_0',real=True, positive=True)
E10_eval = A10*sp.exp(-2*sp.log(2)*t**2/tau**2)*sp.exp(-((x**2+y**2)/w0**2))
E20_eval = A20*sp.exp(-2*sp.log(2)*t**2/tau**2)*sp.exp(-((x**2+y**2)/w0**2))
E30_eval = A30*sp.exp(-2*sp.log(2)*t**2/tau**2)*sp.exp(-((x**2+y**2)/w0**2))
sp.Eq(E10,E10_eval)

Eq(E_{10}(t, x, y), A_{10}*exp(-(x**2 + y**2)/w_0**2)*exp(-2*t**2*log(2)/tau**2))

In [63]:
n1,n2,n3,Xi1,Xi2,Xi3 = sp.symbols('n_1 n_2 n_3 xi_1 xi_2 xi_3')
I1_eval = eps0*c*n1*sp.Abs(E10_eval)**2/2
Xi1_eval = sp.Integral(sp.Integral(sp.Integral(I1_eval,(t,-sp.oo,sp.oo)),(x,-sp.oo,sp.oo)),(y,-sp.oo,sp.oo)).doit()
I2_eval = eps0*c*n2*sp.Abs(E20_eval)**2/2
Xi2_eval = sp.Integral(sp.Integral(sp.Integral(I2_eval,(t,-sp.oo,sp.oo)),(x,-sp.oo,sp.oo)),(y,-sp.oo,sp.oo)).doit()
I3_eval = eps0*c*n3*sp.Abs(E30_eval)**2/2
Xi3_eval = sp.Integral(sp.Integral(sp.Integral(I3_eval,(t,-sp.oo,sp.oo)),(x,-sp.oo,sp.oo)),(y,-sp.oo,sp.oo)).doit()
sp.Eq(Xi1, Xi1_eval)

Eq(xi_1, pi**(3/2)*A_{10}**2*c*epsilon_0*n_1*tau*w_0**2/(8*sqrt(log(2))))

In [64]:
Xias_eval3 = Xias_eval2.subs(E30.subs(t,tp-td),E30_eval).subs(t**2,(tp-td)**2).subs(E10.subs(t,theta),E10_eval.subs(t,theta)).subs(E20.subs(t,theta),E20_eval.subs(t,theta))
sp.Eq(Xias, Xias_eval3)

Eq(\xi_{as}(t_d), Integral(c*epsilon_0*n_{as}*Abs(Integral(-2*I*A_{30}*\kappa_A*omega_3**2*q*exp(-I*(k_3*z' - k_as*z' - omega_3*t' + omega_as*t' + 2*pi))*exp(-(x**2 + y**2)/w_0**2)*exp(-2*(t' - t_d)**2*log(2)/tau**2)/omega_as**2, (z', 0, L)))**2/2, (t', -oo, oo), (x, -oo, oo), (y, -oo, oo)))

In [65]:
Xias_eval4 = Xias_eval3.doit()
Xias_eval4

Piecewise((2**(4*t_d**2/tau**2)*pi**(3/2)*A_{30}**2*c*epsilon_0*n_{as}*omega_3**4*tau*w_0**2*(2 - erfc(2*t_d*sqrt(log(2))/tau))*(I*exp(I*L*k_as)/(k_3*exp(I*L*k_3) - k_as*exp(I*L*k_3)) - I/(k_3 - k_as))*(-I*exp(I*L*k_3)/(k_3*exp(I*L*k_as) - k_as*exp(I*L*k_as)) + I/(k_3 - k_as))*exp(-4*t_d**2*log(2)/tau**2)*Abs(\kappa_A*q)**2/(4*omega_as**4*sqrt(log(2))) + 2**(4*t_d**2/tau**2)*pi**(3/2)*A_{30}**2*c*epsilon_0*n_{as}*omega_3**4*tau*w_0**2*(I*exp(I*L*k_as)/(k_3*exp(I*L*k_3) - k_as*exp(I*L*k_3)) - I/(k_3 - k_as))*(-I*exp(I*L*k_3)/(k_3*exp(I*L*k_as) - k_as*exp(I*L*k_as)) + I/(k_3 - k_as))*exp(-4*t_d**2*log(2)/tau**2)*Abs(\kappa_A*q)**2*erfc(2*t_d*sqrt(log(2))/tau)/(4*omega_as**4*sqrt(log(2))), Ne(k_3, k_as)), (2**(4*t_d**2/tau**2)*pi**(3/2)*A_{30}**2*L**2*c*epsilon_0*n_{as}*omega_3**4*tau*w_0**2*(2 - erfc(2*t_d*sqrt(log(2))/tau))*exp(-4*t_d**2*log(2)/tau**2)*Abs(\kappa_A*q)**2/(4*omega_as**4*sqrt(log(2))) + 2**(4*t_d**2/tau**2)*pi**(3/2)*A_{30}**2*L**2*c*epsilon_0*n_{as}*omega_3**4*tau*w_0**2

In [66]:
Xias_eval5 = (Xias_eval4*(Xi1/Xi1_eval)*(Xi2/Xi2_eval)*(Xi3/Xi3_eval))
Xias_eval5

512*xi_1*xi_2*xi_3*Piecewise((2**(4*t_d**2/tau**2)*pi**(3/2)*A_{30}**2*c*epsilon_0*n_{as}*omega_3**4*tau*w_0**2*(2 - erfc(2*t_d*sqrt(log(2))/tau))*(I*exp(I*L*k_as)/(k_3*exp(I*L*k_3) - k_as*exp(I*L*k_3)) - I/(k_3 - k_as))*(-I*exp(I*L*k_3)/(k_3*exp(I*L*k_as) - k_as*exp(I*L*k_as)) + I/(k_3 - k_as))*exp(-4*t_d**2*log(2)/tau**2)*Abs(\kappa_A*q)**2/(4*omega_as**4*sqrt(log(2))) + 2**(4*t_d**2/tau**2)*pi**(3/2)*A_{30}**2*c*epsilon_0*n_{as}*omega_3**4*tau*w_0**2*(I*exp(I*L*k_as)/(k_3*exp(I*L*k_3) - k_as*exp(I*L*k_3)) - I/(k_3 - k_as))*(-I*exp(I*L*k_3)/(k_3*exp(I*L*k_as) - k_as*exp(I*L*k_as)) + I/(k_3 - k_as))*exp(-4*t_d**2*log(2)/tau**2)*Abs(\kappa_A*q)**2*erfc(2*t_d*sqrt(log(2))/tau)/(4*omega_as**4*sqrt(log(2))), Ne(k_3, k_as)), (2**(4*t_d**2/tau**2)*pi**(3/2)*A_{30}**2*L**2*c*epsilon_0*n_{as}*omega_3**4*tau*w_0**2*(2 - erfc(2*t_d*sqrt(log(2))/tau))*exp(-4*t_d**2*log(2)/tau**2)*Abs(\kappa_A*q)**2/(4*omega_as**4*sqrt(log(2))) + 2**(4*t_d**2/tau**2)*pi**(3/2)*A_{30}**2*L**2*c*epsilon_0*n_{as}*om

In [67]:
sp.Eq(kappaA,kappaA_sub)

Eq(\kappa_A, N*\left(\frac{\partial\alpha}{\partial{q}}\right)_0*omega_as/(2*c*epsilon_0*n_{as}))

In [68]:
sp.Eq(kappaQ,kappaQ_sub)

Eq(\kappa_Q, -\left(\frac{\partial\alpha}{\partial{q}}\right)_0/(2*Omega_R*m))

In [69]:
latex_code = sp.latex(kappaQ_sub)
print(latex_code)

- \frac{\left(\frac{\partial\alpha}{\partial{q}}\right)_0}{2 \Omega_{R} m}


In [70]:
print(Xias_eval5)

512*xi_1*xi_2*xi_3*Piecewise((2**(4*t_d**2/tau**2)*pi**(3/2)*A_{30}**2*c*epsilon_0*n_{as}*omega_3**4*tau*w_0**2*(2 - erfc(2*t_d*sqrt(log(2))/tau))*(I*exp(I*L*k_as)/(k_3*exp(I*L*k_3) - k_as*exp(I*L*k_3)) - I/(k_3 - k_as))*(-I*exp(I*L*k_3)/(k_3*exp(I*L*k_as) - k_as*exp(I*L*k_as)) + I/(k_3 - k_as))*exp(-4*t_d**2*log(2)/tau**2)*Abs(\kappa_A*q)**2/(4*omega_as**4*sqrt(log(2))) + 2**(4*t_d**2/tau**2)*pi**(3/2)*A_{30}**2*c*epsilon_0*n_{as}*omega_3**4*tau*w_0**2*(I*exp(I*L*k_as)/(k_3*exp(I*L*k_3) - k_as*exp(I*L*k_3)) - I/(k_3 - k_as))*(-I*exp(I*L*k_3)/(k_3*exp(I*L*k_as) - k_as*exp(I*L*k_as)) + I/(k_3 - k_as))*exp(-4*t_d**2*log(2)/tau**2)*Abs(\kappa_A*q)**2*erfc(2*t_d*sqrt(log(2))/tau)/(4*omega_as**4*sqrt(log(2))), Ne(k_3, k_as)), (2**(4*t_d**2/tau**2)*pi**(3/2)*A_{30}**2*L**2*c*epsilon_0*n_{as}*omega_3**4*tau*w_0**2*(2 - erfc(2*t_d*sqrt(log(2))/tau))*exp(-4*t_d**2*log(2)/tau**2)*Abs(\kappa_A*q)**2/(4*omega_as**4*sqrt(log(2))) + 2**(4*t_d**2/tau**2)*pi**(3/2)*A_{30}**2*L**2*c*epsilon_0*n_{as}*om